<a href="https://colab.research.google.com/github/Vincent-G-Van/cmpe255-project/blob/main/Project_Predict_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**THIS NOTEBOOK WAS INTENDED AS A PREDICTOR BUT HAS BEEN REPLACED WITH Project_Group_5.ipynb**

This notebook can be used to make predictions on a test set.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def load_json(json_file):
  with open(json_file) as f:
    data = json.load(f)
  return data

def write_json(json_file, data):
  with open(json_file, "w") as outfile:
    json.dump(data, outfile)

In [ ]:
# this function will refactor the data dictionary to a format
# that is more friendly for dataframes
def refactor_data(data_dict):
  # create the dictionary that will be converted to a dataframe
  refactored_data_dict = {}
  # init list of unique authors
  authors = []
  # init raw labels (authors) and data (text)
  # that will be used for actual training
  labels = []
  data = []
  # parse the dictionary to get the files for each author
  for author_entry, file_entry in data_dict.items():
    # init the text list for each author
    text_list = []
    # add the current author to the list of total authors
    authors.append(author_entry)
    # parse the text list to add to the relevant lists
    for file_name, file_text in file_entry.items():
      labels.append(author_entry)
      data.append(file_text)
    refactored_data_dict[author_entry] = data
  data_df = pd.DataFrame(refactored_data_dict, index=None).T
  return data_df, authors, labels, data

In [ ]:
def load_classifier(filename):
  loaded_model = pickle.load(open(filename, 'rb'))
  return loaded_model

In [ ]:
def predict_test_set(classifier_path, vectorizer_path, test_set_json_path):
  loaded_model = load_classifier(classifier_path)

  test_json = load_json(test_set_json_path)
  test_data_df, test_authors, test_labels, test_data = refactor_data(test_json)

  test_labels_np = np.array(test_labels)
  test_data_np = np.array(test_data)

  vect = TfidfVectorizer(stop_words='english')
  vect = pickle.load(open(vectorizer_path, 'rb'))
  test_data_vect = vect.fit_transform(test_data_np)

  predictions = loaded_model.predict(test_data_vect)
  rep = classification_report(test_labels_np, predictions)
  print(rep)
  return predictions

In [ ]:
# read the jsons
test_json_path = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/C50test.json'
classifier_path = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/logistic_regression_model.pkl'
vectorizer_filename = '/content/drive/MyDrive/Assignments/03: FA24: CMPE-255 Sec 33 - Data Mining/Project/vectorizer.pkl'
test_json = predict_test_set(classifier_path=classifier_path,
                             vectorizer_path=vectorizer_filename,
                             test_set_json_path=test_json_path)

ValueError: X has 29786 features, but LogisticRegression is expecting 27092 features as input.